### Task 104: Add graph of token balances for each holder

In [1]:
import os
import copy
import numpy as np
import time
import matplotlib.pyplot as plt
cwd =  os.getcwd().replace("notebooks/test","")
os.chdir(cwd)

from python.dev.event import Deposit
from python.dev.event import Withdraw
from python.dev.cpt import Liquidity
from python.dev.token import RebaseToken
from python.dev.math.model import TimeDeltaModel
from python.dev.math.model import TokenDeltaModel

In [2]:
time0 = time.time()
N_TOKENS = 4
N_ADDR = 5

In [3]:
APY = 0.1
TYPE_DEPOSIT_TOKEN = 'DEPOSIT'
TOKEN = N_TOKENS * [None]
ADDR = N_TOKENS*N_ADDR*[None]
deposit = 10000 

for k in range(N_TOKENS):
    TOKEN[k] = RebaseToken(TYPE_DEPOSIT_TOKEN)
    for j in range(N_ADDR):
        ADDR[j + N_ADDR*k] = TOKEN[k].gen_address()
        TOKEN[k].init_token(time0, ADDR[j + N_ADDR*k])
        TOKEN[k].add_event(Deposit(APY, deposit, 0, ADDR[j + N_ADDR*k]))
   

In [4]:
percent = 0.05
N = 100

for n in range(N):
        
    for k in range(N_TOKENS):
        for j in range(N_ADDR):    
            token_delta = TokenDeltaModel(1,100).apply() 
            time_delta = TimeDeltaModel().apply() 
            TOKEN[k].add_event(Deposit(APY, token_delta, time_delta, ADDR[j + N_ADDR*k]))
    
    if (n % int(N/(100*percent)) == 0):
        out_str = '[SIM {}]'.format(n)
        for k in range(N_TOKENS):
            amt = TOKEN[k].get_balance_deposits()
            out_str = out_str+' T{}: {:.2f}'.format(k+1,amt)
            
        print(out_str) 
        

[SIM 0] T1: 50328.18 T2: 50678.70 T3: 50310.87 T4: 50302.53
[SIM 20] T1: 60489.99 T2: 59181.31 T3: 58693.02 T4: 61169.22
[SIM 40] T1: 72440.80 T2: 68948.32 T3: 69697.98 T4: 70933.19
[SIM 60] T1: 81077.72 T2: 79211.15 T3: 80302.21 T4: 82089.51
[SIM 80] T1: 92295.47 T2: 89970.14 T3: 91470.61 T4: 93177.15


In [5]:
T_states = N_TOKENS*N_ADDR*[None]
final_balances = N_TOKENS*N_ADDR*[0]

fig, (T_ax) = plt.subplots(nrows=1, sharex=True, sharey=False, figsize=(12, 10))

for k in range(N_TOKENS):
    for j in range(N_ADDR):
        m = j + N_ADDR*k
        T_states[m] = TOKEN[k].get_state_series(ADDR[m])
        tSamp = T_states[m].get_tstamp() 
        balance = T_states[m].get_balance() 
        final_balances[m] = balance[-1]
        T_ax.plot(tSamp, balance, label='T{} {}'.format(k,ADDR[m])) 

T_ax.set_title("Token Balances (rebasing token)", fontsize=20)
T_ax.set_ylabel("balance", fontsize=16)
T_ax.set_xlabel('time', fontsize=16)
T_ax.legend(fontsize=8)
        

In [6]:
bar_width = 0.15
x = np.arange(N_TOKENS)
token_names = ['T'+str(k) for k in range(N_TOKENS)]

fig, (ax1) = plt.subplots(nrows=1, sharex=True, sharey=False, figsize=(12, 8))
ax1.set_title("Final token balances", fontsize=20)
ax1.set_ylabel('Balances', fontsize=16)

for k in range(N_ADDR):
    ax1.bar(x + k*bar_width, final_balances[k:20+k:5], width=bar_width, label='address {}'.format(k))

ax1.set_xticks(x + 0.3)
ax1.set_xticklabels(token_names)
plt.legend()
plt.show()